Import de CSV et traitement de la colonne des virement pour séparer les informations 

In [37]:
import csv 
from datetime import date 

today = date.today()
FILENAME = "relevé3.csv"

#Add OS package to catégorize csv file so i don't have to find them 
#Step by step : search for the csv file, find it, do the treatment, put the file in an othe directory with standard name and date. 
# If no csv files in current dir, take the last csv file treated

def traitement(text : str) -> (str,str):
    if "ACHAT CB" in text : 
        text = text.replace("ACHAT CB", "")
        destinataire = text[:-43].strip()
        return destinataire.replace('"', "").strip(), "CB"
    
    elif "PRELEVEMENT DE" in text : 
        text = text.replace("PRELEVEMENT DE", "")
        destinataire = text.split("REF")[0]
        return destinataire.replace('"', "").strip(), "PRELEVEMENT"
    
    elif "VIREMENT INSTANTANE A" in text :
        text = text.replace('VIREMENT INSTANTANE A', "")
        destinataire = text
        return destinataire.replace('"', "").strip(), "VIREMENT SORTANT"
    
    elif "VIREMENT INSTANTANE DE" in text : 
        text = text.replace('VIREMENT INSTANTANE DE', "")
        destinataire = text
        return destinataire.strip(), "VIREMENT ENTRANT"
    
    else : return "LIQUIDE", "RETRAIT"

Retourne la catégorie en fonction du destinataire 

In [38]:
categories = {
            "TRANSPORT" : ['SNCF', 'SNCF INTERNET', 'SNCF WEB MOBIL'], 
            "NOURRITURE" : ["PICARD SA", "FRANPRIX 5141", "MC DONALD S", "CARREFOUR", 'SC-BOUL CHARRI', 'IZLY SMONEY', 'AUCHAN SUPERMA', 'MAISON ISAAC','L ORIENT EXPRE', 'BOUL. MARQUETT'],
            "UTC" : ["UTC", "WEEZEVENT", 'BDE UTC PAYUTC', "WEEZEVENT CASH"], 
            "Réguliers" : ['EDF clients parti iers', "MATMUT ROUEN", "Orange SA"], 
            "AUTRE" : [""]
            }

def categorie(destinataire : str) -> str : 
    for i in categories.keys() :
        if destinataire in categories[i] : 
            return i 
    return "AUTRE"

Pour chaque ligne, on effectue le traitement et retourne la ligne traitée


In [39]:
def traitement_de_ligne(row) : 
    destinataire, moyen_de_payement = traitement(row[1])
    row[1] = destinataire
    row.append(moyen_de_payement)

    float_montant = float(row[2].replace(",","."))
    row[2] = float_montant

    row.append(categorie(destinataire))
    return row

On ouvre le fichier et on applique le traitement pour chaques ligne, puis on ajoute chaque ligne au fichier

In [40]:
with open(FILENAME) as csvfile : 
    csvreader = csv.reader(csvfile, delimiter=";")

    header = [next(csvreader) for i in range(7)]
    new_file = []
    for row in csvreader : 

        row = traitement_de_ligne(row)
        new_file.append(row)
        print(row)

['02/12/2022', 'LIQUIDE', 260.0, 'RETRAIT', 'AUTRE']
['01/12/2022', 'ELIAS REVOLUT Retour des 10euros d urgence', -10.0, 'VIREMENT SORTANT', 'AUTRE']
['01/12/2022', 'PHARMACIE DES', -2.95, 'CB', 'AUTRE']
['29/11/2022', 'Orange SA', -19.99, 'PRELEVEMENT', 'Réguliers']
['28/11/2022', 'SNCF', -42.5, 'CB', 'TRANSPORT']
['28/11/2022', 'SNCF WEB MOBIL', -15.0, 'CB', 'TRANSPORT']
['28/11/2022', 'CARREFOUR', -12.52, 'CB', 'NOURRITURE']
['28/11/2022', 'SNCF WEB MOBIL', -7.7, 'CB', 'TRANSPORT']
['28/11/2022', 'SNCF WEB MOBIL', -7.7, 'CB', 'TRANSPORT']
['28/11/2022', 'M LORGNIER AURELIEN OU', 80.0, 'VIREMENT ENTRANT', 'AUTRE']
['25/11/2022', 'Elias Lorgnier Sent from Revolut', 10.0, 'VIREMENT ENTRANT', 'AUTRE']
['23/11/2022', 'CARREFOUR', -2.03, 'CB', 'NOURRITURE']
['21/11/2022', 'CLARA SL Pret', -6.0, 'VIREMENT SORTANT', 'AUTRE']
['21/11/2022', 'EDF clients parti iers', -30.33, 'PRELEVEMENT', 'Réguliers']
['21/11/2022', 'MC DONALD S', -12.0, 'CB', 'NOURRITURE']
['21/11/2022', 'SNCF WEB MOBIL', -

on crée l'objet "sheet" du package openpyxlet on y ajoute la première ligne

In [41]:
import openpyxl

firstline = header[-1]
firstline[1] = "Destinataire"
firstline[2] = "Montant"
firstline += ["moyen de payement","Catégorie"]

wb = openpyxl.Workbook()
sheet = wb.active
sheet.append(firstline) 

pour chaque ligne du nouveau fichier, on l'ajoute à l'objet sheet si elle correspond à une dépense.

In [42]:
for i in new_file : 
    if i[2] < 0 and i[1] != "ELIAS REVOLUT Transition" and i[1] != "ELIAS REVOLUT":
        sheet.append(i)

Création d'une feuille ou les dépenses sont réunies par catégories, puis d'un graphe qui représente le montant des dépenses par catégorie

In [43]:
wb.create_sheet(title="Par Catégorie")
sheet3 = wb["Par Catégorie"]


cat = categories.keys()
sheet3.append(list(cat))
sheet3.append( [sum([abs(n[2]) for n in new_file if n[2]<0 and n[4] == i and n[1] != "ELIAS REVOLUT Transition" and n[1] != "ELIAS REVOLUT"]) for i in cat ])

#graphe par catégorie (trouver comment mettre les catégories en abcisse)
chart_ref = openpyxl.chart.Reference(sheet3,min_col=sheet.min_column,max_col=sheet3.max_column,min_row=2,max_row=sheet3.max_row )
chart_series = openpyxl.chart.Series(chart_ref)
chart = openpyxl.chart.BarChart()
chart.append(chart_series)
sheet3.add_chart(chart,"F5")

Création d'une feuille ou les dépenses sont réunies par jours 

In [44]:
wb.create_sheet(title="Par jour")
sheet2 = wb["Par jour"]

#trouver comment ajouter toutes les dates du mois
"""
1. créer une liste de toutes les dates du moi donné
2. faire la meme fonction pour toutes les dates """

dates = sorted(list(set([i[0] for i in new_file])))
for i in dates :
    sheet2.append([i, abs(sum([n[2] for n in new_file if n[2] < 0 and n[0] == i]))])

wb.save(f'./relevés/relevé{today}.xlsx')   

Adapter la taille des colonnes automatiquement 
Trouver comment mettre la legende pour les charts 